In [1]:

import matplotlib.pyplot as plt
import cv2
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [2]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rescale=1/255 )

In [3]:
image_gen.flow_from_directory('train_data', shuffle=True, batch_size=32,target_size=(24,24),class_mode='binary')

Found 7553 images belonging to 2 classes.


In [4]:
image_gen.flow_from_directory('test_data', shuffle=True, batch_size=32,target_size=(24,24),class_mode='binary')

Found 194 images belonging to 2 classes.


In [5]:
image_shape = (24,24,3)

In [6]:
from keras.models import Sequential
from keras import regularizers
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(24,24,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(24,24,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(24,24,1), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary, 0=cat , 1=dog
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [8]:
batch_size = 32

train_image_gen = image_gen.flow_from_directory('train_data',color_mode='grayscale',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 7553 images belonging to 2 classes.


In [9]:
test_image_gen = image_gen.flow_from_directory('test_data',color_mode='grayscale',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 194 images belonging to 2 classes.


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
train_image_gen.class_indices

{'with_mask': 0, 'without_mask': 1}

In [12]:
test_image_gen.class_indices

{'with_mask': 0, 'without_mask': 1}

In [13]:
SPE= len(train_image_gen.classes)//batch_size
VS = len(test_image_gen.classes)//batch_size
results = model.fit_generator(train_image_gen,epochs=10,
                              steps_per_epoch=SPE,
                              validation_data=test_image_gen,
                             validation_steps=VS)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
236/236 [==============================] - 24s 100ms/step - loss: 0.6059 - accuracy: 0.6536 - val_loss: 0.3311 - val_accuracy: 0.8802
Epoch 2/10
236/236 [==============================] - 23s 97ms/step - loss: 0.3855 - accuracy: 0.8290 - val_loss: 0.1984 - val_accuracy: 0.9219
Epoch 3/10
236/236 [==============================] - 23s 98ms/step - loss: 0.3269 - accuracy: 0.8593 - val_loss: 0.1782 - val_accuracy: 0.9323
Epoch 4/10
236/236 [==============================] - 25s 106ms/step - loss: 0.2877 - accuracy: 0.8790 - val_loss: 0.1624 - val_accuracy: 0.9375
Epoch 5/10
236/236 [==============================] - 23s 99ms/step - loss: 0.2504 - accuracy: 0.8967 - val_loss: 0.1270 - val_accuracy: 0.9375
Epoch 6/10
236/236 [==============================] - 24s 100ms/step - loss: 0.2161 - accuracy: 0.9107 - val_loss: 0.2172 - val_accuracy: 0.9062
Epoch 7/10
236/236 [==============================] - 23s

In [7]:
from keras.models import load_model
model = load_model('model-010.h5')
model.save('model-010.h5', overwrite=True)

In [8]:
import numpy as np
from keras.preprocessing import image

file = 'test_data/with_mask/111-with-mask.jpg'

img = image.load_img(file,target_size=(24,24), color_mode='grayscale')

img = image.img_to_array(img)

img = np.expand_dims(img, axis=0)
img = img/255
img.shape

(1, 24, 24, 1)

In [9]:
prediction_class = model.predict_classes(img)
print(f'Image belongs to the class: {prediction_class} ')


Image belongs to the class: [[0]] 


In [ ]:
'''

import cv2
import numpy as np
from keras.models import load_model
model=load_model("model-010.h5")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()
'''

In [12]:
import cv2
import os
from keras.models import load_model
import numpy as np
import time


face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

lbl=['with mask','without mask']

model = load_model('model-010.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL


while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 
    time.sleep(0.25)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        
        img=frame[y:y+h,x:x+w]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img,(24,24))
        img= img/255
        img=  img.reshape(24,24,-1)
        img = np.expand_dims(img,axis=0)
        pred = model.predict_classes(img)
        if(pred[0]==1):
            cv2.putText(frame,"without mask",(10,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
        if(pred[0]==0):
            cv2.putText(frame,"with mask",(10,height-20), font, 1,(0,255,0),1,cv2.LINE_AA)
    cv2.imshow('LIVE',   frame)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break 
    
cap.release()
cv2.destroyAllWindows()